# Fine-Tuning T5 for Financial QA and Logging to JFrog ML Registry

This notebook demonstrates fine-tuning a T5 model on financial Q&A data and logging the model version to JFrog ML Registry.

**Key Steps:**
1. **Setup**: Import libraries and define configuration
2. **Data Preparation**: Load and format financial Q&A dataset  
3. **Model Fine-Tuning**: Train T5 using transformers directly
4. **Evaluation**: Test the fine-tuned model with sample questions
5. **Model Logging**: Log the model version to JFrog ML Registry

## 🛠️ Setup and Imports

We'll import all the libraries we need for T5 fine-tuning and model logging.

In [3]:
import pandas as pd
import torch
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset
import frogml

print("✅ Libraries imported successfully!")
print(f"🎯 Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

INFO:datasets:TensorFlow version 2.19.0 available.


✅ Libraries imported successfully!
🎯 Device: CPU


## ⚙️ Model Configuration

Define all the parameters for our T5 model and training process. You can adjust these values to experiment with different settings.

In [10]:
# Model and dataset configuration
model_id = "t5-base"
dataset_name = "financial_qa_dataset"

# Training parameters
config = {
    "model_id": "t5-base",
    "max_length": 128,  # Shorter sequences = faster training
    "target_length": 64,  # Shorter outputs = faster training
    "train_batch_size": 2,  # Smaller batch for CPU
    "eval_batch_size": 2,
    "learning_rate": 5e-4,  # Higher LR for faster convergence
    "num_epochs": 1,  # Just 1 epoch for quick demo
    "max_samples": 10,  # Only use 100 samples total
    "seed": 42,
    "dataset_url": "https://qwak-public.s3.amazonaws.com/example_data/financial_qa.csv"
}

print("✅ Configuration set!")
for key, value in config.items():
    print(f"📊 {key}: {value}")

✅ Configuration set!
📊 model_id: t5-base
📊 max_length: 128
📊 target_length: 64
📊 train_batch_size: 2
📊 eval_batch_size: 2
📊 learning_rate: 0.0005
📊 num_epochs: 1
📊 max_samples: 10
📊 seed: 42
📊 dataset_url: https://qwak-public.s3.amazonaws.com/example_data/financial_qa.csv


## 📊 Data Loading and Preparation

Load the financial Q&A dataset and format it for T5 training. We'll add the "question:" and "answer:" prefixes that T5 expects.

In [11]:
# Load and prepare data
print("📊 Loading financial QA dataset...")
df = pd.read_csv(config["dataset_url"])

# Take a very small subset for quick demo (1-2 minutes training)
df = df.head(config["max_samples"])
print(f"🚀 Using {len(df)} samples for quick demo")

# Format for T5
df['input_text'] = "question: " + df['instruction']
df['target_text'] = "answer: " + df['output']

print(f"✅ Dataset loaded: {df.shape}")
print("\n🔍 Sample:")
print(f"Input: {df['input_text'].iloc[0]}")
print(f"Target: {df['target_text'].iloc[0]}")

📊 Loading financial QA dataset...
🚀 Using 10 samples for quick demo
✅ Dataset loaded: (10, 7)

🔍 Sample:
Input: question: For a car, what scams can be plotted with 0% financing vs rebate?
Target: answer: The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get 

## 🚀 Model Training

Load the T5 model, tokenize our data, and start the fine-tuning process. This will take a few minutes depending on your hardware.

In [12]:
# Set random seed
torch.manual_seed(config["seed"])
np.random.seed(config["seed"])

# Load tokenizer and model
print("🔧 Loading T5 model and tokenizer...")
tokenizer = T5Tokenizer.from_pretrained(config["model_id"])
model = T5ForConditionalGeneration.from_pretrained(config["model_id"])

# Tokenize the data
print("🔧 Tokenizing dataset...")
def tokenize_function(examples):
    inputs = tokenizer(examples["input_text"], max_length=config["max_length"], truncation=True, padding="max_length")
    targets = tokenizer(examples["target_text"], max_length=config["target_length"], truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Convert to Hugging Face dataset format
from datasets import Dataset as HFDataset
dataset = HFDataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split into train/eval
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=config["seed"])
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

print(f"📊 Training samples: {len(train_dataset)}")
print(f"📊 Eval samples: {len(eval_dataset)}")

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=config["num_epochs"],
    per_device_train_batch_size=config["train_batch_size"],
    per_device_eval_batch_size=config["eval_batch_size"],
    learning_rate=config["learning_rate"],
    logging_steps=10,  # Log more frequently for small dataset
    save_steps=1000,  # Don't save checkpoints for quick demo
    eval_steps=50,  # Evaluate more frequently
    save_total_limit=1,  # Save space
    dataloader_num_workers=0,  # Avoid multiprocessing on CPU
    fp16=False,  # Ensure CPU compatibility
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training completed!")

🔧 Loading T5 model and tokenizer...
🔧 Tokenizing dataset...


Map: 100%|██████████| 10/10 [00:00<00:00, 319.69 examples/s]

📊 Training samples: 8
📊 Eval samples: 2


🚀 Starting training...


/Users/haha/anaconda3/envs/qwak-new-3.11/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


✅ Training completed!


## 🧪 Testing the Fine-tuned Model

Test our newly trained model with some sample financial questions to see how well it learned.

In [14]:
# Test the fine-tuned model
print("🧪 Testing fine-tuned model...")


# Force CPU usage to avoid MPS issues
model = model.to("cpu")

def generate_answer(question, model, tokenizer, max_length=150):
    input_text = f"question: {question}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    with torch.no_grad():
        outputs = model.generate(
            **input_ids,
            max_length=max_length,
            num_beams=2,
            early_stopping=True,
            do_sample=False
        )
    
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Test questions
test_questions = [
    "What is the difference between stocks and bonds?",
    "How does inflation affect interest rates?", 
    "What is compound interest?"
]

print("🎯 Sample responses:")
for i, question in enumerate(test_questions, 1):
    answer = generate_answer(question, model, tokenizer)
    print(f"\n--- Question {i} ---")
    print(f"❓ {question}")
    print(f"💡 {answer}")

print("\n✅ Evaluation completed!")

🧪 Testing fine-tuned model...
🎯 Sample responses:

--- Question 1 ---
❓ What is the difference between stocks and bonds?
💡 what is the difference between stocks and bonds?

--- Question 2 ---
❓ How does inflation affect interest rates?
💡 inflation affect interest rates.

--- Question 3 ---
❓ What is compound interest?
💡 compound interest?

✅ Evaluation completed!


## 📦 Save to JFrog ML Registry

Log our trained model to JFrog ML Registry so we can deploy it later. Make sure to update the file path below.

In [ ]:
# Log model to JFrog ML Registry
print("📦 Logging model to JFrog ML Registry...")

try:
    frogml.huggingface.log_model(   
        model=model,
        tokenizer=tokenizer,
        repository="finance-models",    # The JFrog repository to upload the model to
        model_name="finetuned_financial_qa",     # The uploaded model name
        version="",     # Optional. The uploaded model version
        parameters={"finetuning-dataset": dataset_name, "epochs": config["num_epochs"]},
    )
    print("🎉 Model logged successfully to JFrog ML Registry!")
    
except Exception as e:
    print(f"❌ Error logging model: {e}")

INFO:frogml.sdk.model_version.utils.model_log_config:No version provided; using current datetime as the version
INFO:HuggingfaceModelVersionManager:Logging model finetuned_financial_qa to finance-models


📦 Logging model to JFrog ML Registry...


INFO:JmlCustomerClient:Customer exists in JML.
INFO:JmlCustomerClient:Getting project key for repository finance-models
INFO:frogml.sdk.model_version.utils.files_tools:Code directory, predict file and dependencies are not provided. Setup template files for model_name finetuned_financial_qa
/private/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmptpojhukr/finetuned_financial_qa.pretrained_model/special_tokens_map.json: 100%|██████████| 2.54k/2.54k [00:00<00:00, 6.04kB/s]


/private/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmptpojhukr/finetuned_financial_qa.pretrained_model/special_tokens_map.json: 100%|██████████| 2.54k/2.54k [00:00<00:00, 2.98kB/s]



/private/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmptpojhukr/finetuned_financial_qa.pretrained_model/generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]


/private/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmptpojhukr/finetuned_financial_qa.pretrained_model/added_tokens.json: 100%|██████████| 2.59k/2.

2025-09-19 20:43:12,819 - INFO - frogml.storage.logging._log_config.frog_ml.__upload_model:540 - Model: "finetuned_financial_qa", version: "2025-09-19-17-41-57-120" has been uploaded successfully


🎉 Model logged successfully to JFrog ML Registry!


## ✅ Training Workflow Complete!

### 🎯 What we accomplished:
1. ✅ **Fine-tuned T5** on financial Q&A data locally
2. ✅ **Tested model** with sample financial questions  
3. ✅ **Logged the tokenizer and model binaries to JFrog ML Registry** with complete metadata
4. ✅ **Ready for production deployment** via JFrogML platform

### 🚀 Your Model is Now Ready For:
- **Container Building**: Automated packaging in JFrogML UI
- **Real-time Serving**: Instant financial Q&A API endpoints
- **Batch Processing**: Large-scale financial document analysis
- **A/B Testing**: Compare with other model versions
- **Team Collaboration**: Share with colleagues via registry

**🎉 Congratulations!** Your Financial QA model is ready for the next stage of the ML lifecycle! 🚀